In [3]:
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sentence_transformers import SentenceTransformer
import numpy as np

# Cargar modelo BERT multilingüe
model = SentenceTransformer('distiluse-base-multilingual-cased')

# Cargar el archivo Excel
ruta_archivo = 'ArticulosOriginales.xlsx'
df = pd.read_excel(ruta_archivo)

# Asegurar que las columnas necesarias están presentes
assert 'Texto' in df.columns and 'IDArticulo' in df.columns, "El archivo Excel debe contener las columnas 'Texto' e 'IDArticulo'."

# Procesar el texto
def procesar_texto(texto):
    # Borrar los |
    texto = texto.replace('|', '.')
    # Normalizar comillas y >> o <<
    texto = texto.replace('“', '"').replace('”', '"')
    texto = texto.replace('»', '"').replace('«', '"')
    return texto

df['Texto'] = df['Texto'].apply(procesar_texto)

# Filtrar artículos con menos de 4 palabras
df = df[df['Texto'].str.split().str.len() > 3]

# Convertir las frases a embeddings
frases = df['Texto'].tolist()
frases_embed = model.encode(frases, convert_to_tensor=True)

# Convertir tensores a arrays de numpy
frases_embed_np = frases_embed.cpu().numpy()

# Aplicar clustering jerárquico
umbral_distancia = 1.0  # Ajusta este valor si es necesario
clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=umbral_distancia, linkage='average')
etiquetas = clustering.fit_predict(frases_embed_np)

# Añadir la columna de grupos al dataframe
df['IDGrupo'] = etiquetas

# Filtrar grupos que no tienen al menos 20 artículos
df_grouped = df.groupby('IDGrupo').filter(lambda x: len(x) >= 20)

# Reordenar columnas para el output
df_output = df_grouped[['IDGrupo', 'Texto', 'IDArticulo']]

# Guardar el nuevo dataframe en un archivo Excel
df_output.to_excel('ArticulosAgrupados.xlsx', index=False)

c:\Anaconda\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
